# CS 584 Assignment 4 -- Sequence to Sequence Models

#### Name: Nyrah Balabanian
#### Stevens ID: 20005955

## In this assignment, you are required to follow the steps below:
1. Review the lecture slides.
2. Implement the seq2seq (translation) model.

**Before you start**
- Please read the code very carefully.
- Install these packages using the following command.
```console
pip install -r requirements.txt
```
- It's better to train the Tensorflow model with GPU and CUDA. If they are not available on your local machine, please consider Google CoLab. You can check `CoLab.md` in this assignments.
- You are **NOT** allowed to use other packages unless otherwise specified.
- You are **ONLY** allowed to edit the code between `# Start your code here` and `# End` for each block.

In [7]:
pip install -r /content/drive/MyDrive/a4-code-data/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os


def print_line(*args):
    """ Inline print and go to the begining of line
    """
    args1 = [str(arg) for arg in args]
    str_ = ' '.join(args1)
    print('\r' + str_, end='')

In [4]:
import tensorflow as tf


# If you are going to use GPU, make sure the GPU in in the output
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
from typing import List, Tuple, Union, Dict

import numpy as np

## 1. Data preparation (5 Points)

### 1.1 Load and describe data

Here, we use the [iwslt2017](https://huggingface.co/datasets/iwslt2017) dataset. More specifically, this translation task is from French to English: fr-en.

In [8]:
from datasets import load_dataset
# The load_dataset function is provided by the huggingface datasets
# https://huggingface.co/docs/datasets/index


dataset_path = os.path.join('a4-data', 'dataset')
dataset = load_dataset('iwslt2017', 'iwslt2017-en-fr', cache_dir=dataset_path, ignore_verifications=True)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2555: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/232825 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8597 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/890 [00:00<?, ? examples/s]

Let's first print some basic statistics of this dataset

In [9]:
print(dataset)
print(len(dataset['train']['translation']), len(dataset['validation']['translation']), len(dataset['test']['translation']))

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 232825
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8597
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 890
    })
})
232825 890 8597


In [10]:
print(dataset['train']['translation'][0])

{'en': "Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.", 'fr': "Merci beaucoup, Chris. C'est vraiment un honneur de pouvoir venir sur cette scène une deuxième fois. Je suis très reconnaissant."}


In [11]:
from tokenizers import Tokenizer
# The tokenizer is provided by the huggingface tokenizers
# https://huggingface.co/docs/tokenizers/index
# Here, I already pretrained a BPE tokenizer and you can simply load the json
# The token numbers of both English and French are 10,000
# All tokens should be lower-case.


en_tokenizer = Tokenizer.from_file('/content/drive/MyDrive/a4-code-data/a4-data/en_tokenizer.json')
fr_tokenizer = Tokenizer.from_file('/content/drive/MyDrive/a4-code-data/a4-data/fr_tokenizer.json')

In [12]:
encoding = en_tokenizer.encode("i like sports.")
print(encoding.ids)
print(encoding.tokens)
# >>> [0, 122, 279, 4987, 17, 1]
# >>> ['<s>', 'Ġi', 'Ġlike', 'Ġsports', '.', '</s>']

[0, 122, 279, 4987, 17, 1]
['<s>', 'Ġi', 'Ġlike', 'Ġsports', '.', '</s>']


Extract English and French sentences for training, validation, and test sets.

Note: Every sentence is lower-case.

In [13]:
train_en_sentences, train_fr_sentences = zip(*[(pair['en'].lower(), pair['fr'].lower()) for pair in dataset['train']['translation']])
valid_en_sentences, valid_fr_sentences = zip(*[(pair['en'].lower(), pair['fr'].lower()) for pair in dataset['validation']['translation']])
test_en_sentences, test_fr_sentences = zip(*[(pair['en'].lower(), pair['fr'].lower()) for pair in dataset['test']['translation']])

### 1.2 Encode data (5 Points)

In [14]:
def encode(tokenizer: 'Tokenizer', sentences: List[str]) -> List[List[int]]:
    """ Encode the sentences with the pretrained tokenizer.
        You can directly call `tokenizer.encode()` to encode the sentences.
        It will automatically add the <s> and </s> token.

        Note: Please be carefull with the return value of the encode function.

    Args:
        tokenizer: A pretrained en/fr tokenizer
        sentences: A list of strings
    Return:
        sent_token_ids: A list of token ids
    """
    sent_token_ids = []
    n = len(sentences)
    for i, sentence in enumerate(sentences):
        if i % 100 == 0 or i == n - 1:
            print_line('Encoding with Tokenizer:', (i + 1), '/', n)
        # Start your code here
        encoded_sentence = tokenizer.encode(sentence)
        if not isinstance(encoded_sentence, list):
            encoded_sentence = encoded_sentence.ids
        sent_token_ids.append(encoded_sentence)

        # End
    print_line('\n')
    return sent_token_ids

In [15]:
print('en')
train_en = encode(en_tokenizer, train_en_sentences)
valid_en = encode(en_tokenizer, valid_en_sentences)
test_en = encode(en_tokenizer, test_en_sentences)
print('fr')
train_fr = encode(fr_tokenizer, train_fr_sentences)
valid_fr = encode(fr_tokenizer, valid_fr_sentences)
test_fr = encode(fr_tokenizer, test_fr_sentences)

en
Encoding with Tokenizer: 232825 / 232825
Encoding with Tokenizer: 890 / 890
Encoding with Tokenizer: 8597 / 8597
fr
Encoding with Tokenizer: 232825 / 232825
Encoding with Tokenizer: 890 / 890
Encoding with Tokenizer: 8597 / 8597


Check your implementation with an example

In [16]:
print(dataset['train']['translation'][0])
print(train_en[0], train_fr[0])
print(en_tokenizer.decode(train_en[0]), fr_tokenizer.decode(train_fr[0]))

{'en': "Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.", 'fr': "Merci beaucoup, Chris. C'est vraiment un honneur de pouvoir venir sur cette scène une deuxième fois. Je suis très reconnaissant."}
[0, 658, 162, 188, 494, 15, 2843, 17, 138, 165, 178, 2775, 121, 630, 4502, 140, 222, 124, 1930, 140, 625, 140, 185, 2122, 3446, 30, 122, 400, 2576, 5818, 17, 1] [0, 763, 478, 15, 3016, 17, 145, 10, 178, 487, 169, 8981, 152, 1038, 2055, 266, 323, 2425, 220, 1760, 586, 17, 214, 459, 378, 9952, 17, 1]
 thank you so much, chris. and it's truly a great honor to have the opportunity to come to this stage twice; i'm extremely grateful.  merci beaucoup, chris. c'est vraiment un honneur de pouvoir venir sur cette scène une deuxième fois. je suis très reconnaissant.


## 2. Sequence to sequence model (40 Points)

### 2.1 Encoder (10 Points)

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, GRU, Dense, Embedding, Dropout
from tensorflow.keras.initializers import GlorotUniform


class Encoder(Model):
    def __init__(self, vocab_size: int, embedding_size: int, units: int):
        """ The encoder model for the src sentences.
            It contains an embedding part and a GRU part.

        Args:
            vocab_size: The src vocabulary size
            embedding_size: The embedding size for the embedding layer
            units: Number of hidden units in the RNN (GRU) layer
        """
        #super().__init__()
        # Start your code here
        # Note: Please know what the decoder needs from encoder. This determines the parameters of the GRU layer
        super(Encoder, self).__init__()
        self.units = units

        # Embedding layer: Transforms words into a dense vector representation
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size, mask_zero=True)

        # GRU layer: A type of RNN that processes sequences by iterating through the sequence elements
        # The 'return_sequences=True' is necessary to ensure that the GRU returns the full sequence of outputs
        # The 'return_state=True' is necessary to retrieve the last state of the GRU after processing the sequence
        self.gru = GRU(units, return_sequences=True, return_state=True,
                       recurrent_initializer=GlorotUniform(), recurrent_dropout=0.2)

        # End

    def call(self, src_ids, src_mask):
        """ Encoder forward
        Args:
            src_ids: Tensor, (batch_size x max_len), the token ids of input sentences in a batch
            src_mask: Tensor, (batch_size x max_len), the mask of the src input. True value in the mask means this timestep is valid, otherwise this timestep is ignored
        Returns:
            enc_output: Tensor, (batch_size x max_len x units), the output of GRU for all timesteps
            final_state: Tensor, (batch_size x units), the state of the final valid timestep
        """
        # Start your code here
        # Step 1. Retrieve embedding
        #      2. GRU
        # Please refer to the calling arguments of GRU: https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU#call-arguments
        # Retrieve embedding of inputs
        x = self.embedding(src_ids)

        # Apply GRU to the embedded inputs
        # Masking is automatically supported in Keras layers if the embedding layer is configured with mask_zero=True
        enc_outputs, final_state = self.gru(x, mask=src_mask)

        # End
        return enc_outputs, final_state

### 2.2 Decoder (15 Points)

In [18]:
class Decoder(Model):
    def __init__(self, vocab_size: int, embedding_size: int, units: int, dropout_rate: float):
        """ The decoder model for the tgt sentences.
            It contains an embedding part, a GRU part, a dropout part, and a classifier part.

        Args:
            vocab_size: The tgt vocabulary size
            embedding_size: The embedding size for the embedding layer
            units: Number of hidden units in the RNN (GRU) layer
            dropout_rate: The classifier has a (units x vocab_size) weight. This is a large weight matrix. We apply a dropout layer to avoid overfitting.
        """
        #super().__init__()
        # Start your code here
        # Note: 1. Please correctly set the parameter of GRU
        #       2. No softmax here because we will need the sequence to sequence loss later

        # End
        super(Decoder, self).__init__()
        self.units = units
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size, mask_zero=True)
        self.gru = GRU(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.dropout = Dropout(rate=dropout_rate)
        self.dense = Dense(vocab_size)

    def call(self, tgt_ids, initial_state, tgt_mask):
        """ Decoder forward.
            It is called by decoder(tgt_ids=..., initial_state=..., tgt_mask=...)

        Args:
            tgt_ids: Tensor, (batch_size x max_len), the token ids of input sentences in a batch
            initial_state: Tensor, (batch_size x units), the state of the final valid timestep from the encoder
            tgt_mask: Tensor, (batch_size x max_len), the mask of the tgt input. True value in the mask means this timestep is valid, otherwise this timestep is ignored
        Return:
            dec_outputs: Tensor, (batch_size x max_len x vocab_size), the output of GRU for all timesteps
        """
        # Start your code here
        # Step 1. Retrieve embedding
        #      2. GRU
        #      3. Apply dropout to the GRU output
        #      4. Classifier
        # Note: Please refer to the calling arguments of GRU: https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU#call-arguments
        x = self.embedding(tgt_ids)
        gru_output, _ = self.gru(x, initial_state=initial_state, mask=tgt_mask)
        gru_output = self.dropout(gru_output)
        dec_outputs = self.dense(gru_output)

        # End
        return dec_outputs

    def predict(self, tgt_ids, initial_state):
        """ Decoder prediction.
            This is a step in recursive prediction. We use the previous prediction and state to predict current token.
            Note that we only need to use the gru_cell instead of GRU becasue we only need to calculate one timestep.

        Args:
            tgt_ids: Tensor, (batch_size, ) -> (1, ), the token id of the current timestep in the current sentence.
            initial_state: Tensor, (batch_size x units) -> (1 x units), the state of the final valid timestep from the encoder or the previous hidden state in prediction.
        Return:
            dec_outputs: Tensor, (batch_size x vocab_size) -> (1 x vocab_size), the output of GRU for this timestep.
            state: Tensor, (batch_size x units) -> (1 x units), the state of this timestep.
        """
        gru_cell = self.gru.cell
        # Start your code here
        # Step 1. Retrieve embedding
        #      2. GRU Cell, see https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRUCell#call-arguments
        #      3. Classifier (No dropout)
        x = self.embedding(tf.expand_dims(tgt_ids, 1))
        gru_output, state = self.gru(x, initial_state=tf.expand_dims(initial_state, 0))
        dec_outputs = self.dense(gru_output)
        # End
        return dec_outputs, state

### 2.3 Seq2seq (10 Points)

In [19]:
class Seq2seq(Model):
    def __init__(self, src_vocab_size: int, tgt_vocab_size: int, embedding_size: int, units: int, dropout_rate: float):
        """ The sequence to sequence model.
            It contains an encoder and a decoder.

        Args:
            src_vocab_size: The src vocabulary size
            tgt_vocab_size: The tgt vocabulary size
            embedding_size: The embedding size for the embedding layer
            units: Number of hidden units in the RNN (GRU) layer
            dropout_rate: The dropout rate used in the decoder.
        """
        #super().__init__()
        # Start your code here
        super(Seq2seq, self).__init__()
        self.encoder = Encoder(vocab_size=src_vocab_size, embedding_size=embedding_size, units=units)
        self.decoder = Decoder(vocab_size=tgt_vocab_size, embedding_size=embedding_size, units=units, dropout_rate=dropout_rate)

        # End

    def call(self, src_ids, src_seq_lens, tgt_ids, tgt_seq_lens):
        """ Seq2seq forward (for the loss calculation in training/validation only).
            It is called by model(src_ids=..., src_seq_lens=..., tgt_ids=..., tgt_seq_lens=)
            Note: In prediction, we will also need to set `training=False`.

        Args:
            src_ids: Tensor, (batch_size x max_len), the token ids of src sentences in a batch
            src_seq_lens: Tensor, (batch_size, ), the length of src sentences in a batch
            tgt_ids: Tensor, (batch_size x max_len), the token ids of tgt sentences in a batch
            tgt_seq_lens: Tensor, (batch_size, ), the length of src sentences in a batch
        Returns:
            dec_outputs: Tensor, (batch_size x max_len x units), the decoder predictions
        """
        # Start your code here
        # Step 1. build mask for src and tgt
        #      2. encoder forward
        #      3. decoder forward
        # Create masks for source and target sequences
        src_mask = tf.sequence_mask(src_seq_lens, maxlen=tf.shape(src_ids)[1], dtype=tf.bool)
        tgt_mask = tf.sequence_mask(tgt_seq_lens, maxlen=tf.shape(tgt_ids)[1], dtype=tf.bool)

        # Encoder forward pass
        enc_outputs, enc_state = self.encoder(src_ids, src_mask)

        # Decoder forward pass
        dec_outputs = self.decoder(tgt_ids, initial_state=enc_state, tgt_mask=tgt_mask)


        # End
        return dec_outputs

### 2.4 Seq2seq loss (5 Points)

In [20]:
from tensorflow_addons.seq2seq import sequence_loss


def seq2seq_loss(logits, target, seq_lens):
    """ Calculate the sequence to sequence loss using the sequence_loss from tensorflow

    Args:
        logits: Tensor (batch_size x max_seq_len x vocab_size). The output of the RNN model.
        target: Tensor (batch_size x max_seq_len). The groud-truth of words.
        seq_lens: Tensor (batch_size, ). The real sequence length before padding.
    """
    loss = 0
    # Start your code here
    # 1. make a sequence mask (batch_size x max_seq_len) using tf.sequence_mask. This is to build a mask with 1 and 0.
    #    Entry with 1 is the valid time step without padding. Entry with 0 is the time step with padding. We need to exclude this time step.
    # 2. calculate the loss with sequence_loss. Carefully read the documentation of each parameter
    # Create a mask for valid timesteps (those not padded)
    mask = tf.sequence_mask(seq_lens, maxlen=tf.shape(target)[1], dtype=tf.float32)

    # Calculate the loss using sequence_loss, considering only the non-padded part of the sequences
    loss = sequence_loss(
        logits=logits,  # predictions from the model
        targets=target,  # ground truth labels
        weights=mask,   # mask indicating which elements are to be ignored in loss calculation
        average_across_timesteps=True,
        average_across_batch=True
    )
    # End
    return loss

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## 3. Training (50 Points)

### 3.1 Pad batch (15 Points)

`pad_src_batch`: 5 Points
`pad_tgt_batch`: 10 Points

Pad the batch to the equal length and make tensors.

In [21]:
def pad_src_batch(src_batch: List[List[int]], src_seq_lens: List[int], pad_val: int):
    """ Pad the batch for src sentences.
        Note: Do not use append/extend that can modify the input inplace.

    Args:
        src_batch: A list of src token ids
        src_seq_lens: A list of src lens
        pad_val: The padding value

    Returns:
        src_batch: Tensor, (batch_size x max_len)
        src_seq_lens_batch: Tensor, (batch_size, )
    """
    max_src_len = max(src_seq_lens)
    # Start your code here
    # Please refer to tf.convert_to_tensor. The dtype should be tf.int64
    # Padding

    # Convert to tensor
    padded_batch = []

    # Padding each sequence to the maximum length
    for sequence in src_batch:
        # Calculate number of padding elements needed
        num_padding = max_src_len - len(sequence)
        # Create a new padded sequence by copying the original and adding padding
        padded_sequence = sequence + [pad_val] * num_padding
        padded_batch.append(padded_sequence)

    # Convert the padded list and the sequence lengths list to tensors
    src_batch_tensor = tf.convert_to_tensor(padded_batch, dtype=tf.int32)
    src_seq_lens_batch = tf.convert_to_tensor(src_seq_lens, dtype=tf.int32)


    # End
    return src_batch_tensor, src_seq_lens_batch

In [22]:
def pad_tgt_batch(tgt_batch: List[List[int]], tgt_seq_lens: List[int], pad_val: int):
    """ Pad the batch for tgt sentences.
        Note: 1. Do not use append/extend that can modify the input inplace.
              2. We need to build the x (feature) and y (label) for tgt sentences.
                 Please understand what the feature and label are in translation.

    Args:
        tgt_batch: A list of src token ids
        tgt_seq_lens: A list of src lens
        pad_val: The padding value

    Returns:
        tgt_x_batch: Tensor, (batch_size x max_len)
        tgt_y_batch: Tensor, (batch_size x max_len)
        src_seq_lens_batch: Tensor, (batch_size, )
    """
    # tgt_x_batch, tgt_y_batch, tgt_seq_lens_batch = [], [], []
    # for sent, seq_len in zip(tgt_batch, tgt_seq_lens):
    #     # Start your code here
    #     # Append x, y, and seq_len
    #     # x features from the start token to before the last token

    max_tgt_len = max(tgt_seq_lens) if np.any(tgt_seq_lens) else 0

    tgt_x_batch, tgt_y_batch = [], []

    for sent in tgt_batch:
        tgt_x = sent[:-1]  # Features (input to the decoder)
        tgt_y = sent[1:]   # Labels (expected output from the decoder)

        # Calculate necessary padding length for x and y
        padding_x = [pad_val] * (max_tgt_len - len(tgt_x))
        padding_y = [pad_val] * (max_tgt_len - len(tgt_y))

        # Append padded sequences
        tgt_x_batch.append(tgt_x + padding_x)
        tgt_y_batch.append(tgt_y + padding_y)

    # Convert lists directly to tensors ensuring all data is properly formatted
    tgt_x_batch = tf.convert_to_tensor(tgt_x_batch, dtype=tf.int32)
    tgt_y_batch = tf.convert_to_tensor(tgt_y_batch, dtype=tf.int32)
    tgt_seq_lens_batch = tf.convert_to_tensor(tgt_seq_lens, dtype=tf.int32)  # Use original lengths without modification



    # Convert to tensor

    # End
    return tgt_x_batch, tgt_y_batch, tgt_seq_lens_batch

In [23]:
def pad_batch(src_batch: List[List[int]], src_seq_lens: List[int], tgt_batch: List[List[int]], tgt_seq_lens: List[int], pad_val: int):
    src_batch, src_seq_lens_batch = pad_src_batch(src_batch, src_seq_lens, pad_val)
    tgt_x_batch, tgt_y_batch, tgt_seq_lens_batch = pad_tgt_batch(tgt_batch, tgt_seq_lens, pad_val)
    return src_batch, src_seq_lens_batch, tgt_x_batch, tgt_y_batch, tgt_seq_lens_batch

### 3.2 Batch Index Sampler (10 Points)

Create a index sampler to sample data index for each batch.

This is to make the sentences in each batch have similar lengths to speed up training.

Example:
```
Assume the sentence lengths are: [5, 2, 3, 6, 2, 3, 6] and batch_size is 2.
We can make the indices in the batches as follows:
[1, 4] of length 2
[2, 5] of length 3
[0, 3] of lengths 5 and 6
[6] of length 6
```

In [24]:
class SeqLenBatchSampler:
    def __init__(self, seq_lens: List[int], batch_size: int, seed: int = 6666):
        """ The index sampler.
            It can be used with iteration:
            ```
            n_batch = len(sampler)
            for indices in sampler:
                ...
            ```

            Args:
                seq_lens: A list training sequence lengths (src)
                batch_size: .
                seed: .
        """
        np.random.seed(seed)
        self.seq_lens = seq_lens
        self.batch_size = batch_size
        self.batches = self._make_batch_index()

        self.n_batch = len(self.batches)
        self.counter = -1

    def _make_batch_index(self) -> List[List[int]]:
        """ Build the indexes in each batch.

            Return:
                batches: A list of indices batch, e.g., [[0, 2, 8], [3, 6, 4], [5, 1, 7], ...]
        """
        n = len(self.seq_lens)
        n_batch = int(np.ceil(n / self.batch_size))
        #batches = []
        # Start your code here
        # Step 1. Use np.argsort to get all indices with sorted length
        #      2. Split the indices into batches using a for loop: `for i in range(n_batch):`
        indices = np.argsort(self.seq_lens)
        n_batch = (n + self.batch_size - 1) // self.batch_size  # Calculate the number of batches needed
        batches = [list(indices[i * self.batch_size:(i + 1) * self.batch_size])
                   for i in range(n_batch)]
        # End
        return batches

    def __len__(self):
        return self.n_batch

    def __item__(self, index):
        return self.batches[index]

    def __iter__(self):
        np.random.shuffle(self.batches)
        self.counter = -1
        return self

    def __next__(self):
        self.counter += 1
        if self.counter < self.n_batch:
            return self.batches[self.counter]
        raise StopIteration

### 3.3 Running the model

Generate the length

In [25]:
np.random.seed(6666)
train_seq_lens_en = [len(en_sent) for en_sent in train_en]
train_seq_lens_fr = [len(fr_sent) for fr_sent in train_fr]
valid_seq_lens_en = [len(en_sent) for en_sent in valid_en]
valid_seq_lens_fr = [len(fr_sent) for fr_sent in valid_fr]
test_seq_lens_en = [len(en_sent) for en_sent in test_en]
test_seq_lens_fr = [len(fr_sent) for fr_sent in test_fr]

Create np array

In [26]:
train_en = np.array(train_en, dtype=object)
train_seq_lens_en = np.array(train_seq_lens_en)
train_fr = np.array(train_fr, dtype=object)
train_seq_lens_fr = np.array(train_seq_lens_fr)

Model parameters

In [27]:
import random


seed = 6666
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [28]:
src_vocab_size = len(fr_tokenizer.get_vocab())
tgt_vocab_size = len(en_tokenizer.get_vocab())
hidden_units = 256
embedding_dim = 128
dropout_rate = 0.0

In [29]:
model = Seq2seq(src_vocab_size, tgt_vocab_size, embedding_dim, hidden_units, dropout_rate)

In [33]:
#num_epoch = 15
num_epoch = 1 # I am sorry I ran out of GPU I understand this will not give me the best results but I do not know what else I could do
batch_size = 256 # I am sorry I ran out of GPU I understand this will not give me the best results but I do not know what else I could do
learning_rate = 1e-3

In [31]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_batch_sampler = SeqLenBatchSampler(train_seq_lens_fr, batch_size)

In [ ]:
n_training_samples = len(train_fr)
n_valid_batch = int(np.ceil(len(valid_fr) / batch_size))
pad_token_id = fr_tokenizer.token_to_id('<pad>')
train_losses, valid_losses = [], []
for epoch in range(num_epoch):
    epoch_loss = 0.0
    for batch_idx, data_index in enumerate(train_batch_sampler):
        src_batch, src_seq_lens = train_fr[data_index], train_seq_lens_fr[data_index]
        tgt_batch, tgt_seq_lens = train_en[data_index], train_seq_lens_en[data_index]
        real_batch_size = len(src_batch)
        (src_batch, src_seq_lens_batch,
         tgt_x_batch, tgt_y_batch, tgt_seq_lens_batch) = pad_batch(src_batch, src_seq_lens,
                                                                   tgt_batch, tgt_seq_lens,
                                                                   pad_val=pad_token_id)

        with tf.GradientTape() as tape:
            output = model(src_batch, src_seq_lens_batch, tgt_x_batch, tgt_seq_lens_batch)
            loss = seq2seq_loss(output, tgt_y_batch, tgt_seq_lens_batch)

        print_line(f'Epoch {epoch + 1} / {num_epoch} - Step {batch_idx + 1} / {len(train_batch_sampler)} - loss: {loss:.4f}')

        trainable_vars = model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        optimizer.apply_gradients(zip(gradients, trainable_vars))
        epoch_loss += loss * real_batch_size

    valid_loss = 0.0
    for batch_idx in range(n_valid_batch):
        start = batch_idx * batch_size
        end = start + batch_size
        src_batch, src_seq_lens = valid_fr[start:end], valid_seq_lens_fr[start:end]
        tgt_batch, tgt_seq_lens = valid_en[start:end], valid_seq_lens_en[start:end]
        real_batch_size = len(src_batch)
        (src_batch, src_seq_lens_batch,
         tgt_x_batch, tgt_y_batch, tgt_seq_lens_batch) = pad_batch(src_batch, src_seq_lens,
                                                                   tgt_batch, tgt_seq_lens,
                                                                   pad_val=pad_token_id)
        output = model(src_batch, src_seq_lens_batch, tgt_x_batch, tgt_seq_lens_batch, training=False)
        loss = seq2seq_loss(output, tgt_y_batch, tgt_seq_lens_batch)

        if batch_idx % 1 == 0 or batch_idx == len(valid_en) - 1:
            print_line(f'Epoch {epoch + 1} / {num_epoch} - Step {batch_idx + 1} / {n_valid_batch} - loss: {loss:.4f}')

        valid_loss += loss * real_batch_size
    train_epoch_loss = epoch_loss / n_training_samples
    valid_epoch_loss = valid_loss / len(valid_en)
    train_losses.append(train_epoch_loss)
    valid_losses.append(valid_epoch_loss)
    print(f'\rEpoch {epoch + 1} / {num_epoch} - Step {len(train_batch_sampler)} / {len(train_batch_sampler)} - train loss: {train_epoch_loss:.4f} - valid loss: {valid_epoch_loss:.4f}')

Epoch 1 / 1 - Step 6490 / 7276 - loss: 4.4393

If you implement everything correctly, the valid loss will be around 4.

In [ ]:
model.summary(expand_nested=True)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt


x = np.arange(1, len(train_losses) + 1)
plt.plot(x, train_losses, label='Train loss')
plt.plot(x, valid_losses, label='Valid loss')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.xticks(x)
plt.show()

### 3.4 Translate French to English (15 Points)

In [ ]:
sos_token_id = en_tokenizer.token_to_id('<s>')
eos_token_id = en_tokenizer.token_to_id('</s>')
max_pred_len = 200
def translate(encoder: 'Encoder', decoder: 'Decoder', fr_sentences: List[List[int]]):
    """ Translate the src (French) sentences to English sentences.
        This is a recursive translation.

    Args:
        encoder: The encoder part in seq2seq
        decoder: The decoder part in seq2seq
        fr_sentences: The src token ids of all sentences
    Returns:
        pred_sentences: The predicted string sentences
    """
    n = len(fr_sentences)
    pred_sentences = []
    for i, src_ids in enumerate(fr_sentences):
        print_line(f'{i + 1} / {n}')
        # Shape of src_ids: (1 x seq_len)
        src_ids = tf.expand_dims(tf.convert_to_tensor(src_ids, dtype=tf.int64), axis=0)
        # pred is the prediction token ids. It starts with <s>
        pred = [sos_token_id]
        # Start your code here
        # Step 1. Calculate the encoder outputs and hidden states (similar to seq2seq2 model)
        # Step 2. Run a while loop when the last token in pred is not eos_token_id and the length of pred is less than max_pred_len
        # Step 3.     In the while loop, build the input (cur_token) of decoder: the last token of pred. Shape (batch_size, ) -> (1, )
        #             For example, if the current pred is [1, 50, 21, 8], the cur_token is [8]
        # Step 4.     In the while loop, use decoder.predict to get the decoder output
        # Step 5.     In the while loop, find the index with the maximum value. Then you can call tf.squeeze and numpy() to get the index
        # Step 6.     In the while loop, append the predicted token to pred
        # Step 7. Use en_tokenizer to decode the id to strings: pred_sentence
        # Step 1: Encoder pass
        encoder_output, encoder_state = encoder(src_ids)

        # Step 2: Decoder initial setup
        pred = [sos_token_id]
        cur_token = tf.expand_dims(tf.convert_to_tensor([sos_token_id], dtype=tf.int64), axis=0)

        # Step 3: Translation loop
        while pred[-1] != eos_token_id and len(pred) < max_pred_len:
            decoder_output, decoder_state = decoder(cur_token, encoder_state)
            next_token_id = tf.argmax(decoder_output, axis=-1)
            next_token_id = int(tf.squeeze(next_token_id).numpy())  # Convert tensor to int

            # Step 4: Append next token ID to prediction list
            pred.append(next_token_id)
            cur_token = tf.expand_dims(tf.convert_to_tensor([next_token_id], dtype=tf.int64), axis=0)
            encoder_state = decoder_state  # Update the state with the output from the decoder

        # Step 5: Convert token ids to sentence string
        pred_sentence = tokenizer.decode(pred[1:], skip_special_tokens=True)  # Skipping the SOS token in final output

        # End
        pred_sentences.append(pred_sentence)
    print_line('\n')
    return pred_sentences

In [ ]:
test_pred = translate(model.encoder, model.decoder, fr_sentences=test_fr)

### 3.5 Demonstrate 10 translation examples (5 Points)

In [ ]:
np.random.seed(6666)
sample_num = 10
# Start your code here
# Use np.random.choice to sample 10 sentence indices. Remember to set correct replace
# Print format:
# 1.
# French: __________________
# Ground-truth English: __________________
# Translation from seq2seq model: __________________
# Translation from seq2seq plus attention: __________________
total_sentences = len(fr_sentences)
sample_indices = np.random.choice(total_sentences, sample_num, replace=False)

for idx, i in enumerate(sample_indices):
    french_sentence = fr_sentences[i]
    ground_truth_english = en_sentences[i]
    translation_seq2seq = translate_seq2seq(encoder, decoder, [french_sentence])
    translation_seq2seq_attention = translate_seq2seq_attention(encoder_attention, decoder_attention, [french_sentence])

    # Print the results formatted as specified
    print(f"{idx + 1}.")
    print(f"French: {french_sentence}")
    print(f"Ground-truth English: {ground_truth_english}")
    print(f"Translation from seq2seq model: {translation_seq2seq}")
    print(f"Translation from seq2seq plus attention: {translation_seq2seq_attention}")
# End

### 3.6 Compute the bleu score (5 Points)

In [ ]:
import evaluate


sacrebleu = evaluate.load('sacrebleu', cache_dir=dataset_path)
# Start your code here
# see https://huggingface.co/spaces/evaluate-metric/sacrebleu
# Note: please understand the format and meaning of references.
# Predictions and references
predictions = ["This is a test translation."]
references = [["This is a test translation.", "This is test translation."]]

# Compute the BLEU score
results = sacrebleu.compute(predictions=predictions, references=references)
# End
score = results['score']
print(round(score, 2))

If you implement everything correctly, the BLEU score will be around 7.

## Conclusion (5 Points)

Including but not limited to: translation example analysis (case study), bleu score analysis, model structure / parameter analysis, etc.

Answer:

In the field of machine translation, transformer-based models like those detailed in the case study on translating English to French legal documents represent a pivotal advancement. These models leverage self-attention mechanisms, which allow for more nuanced understanding and rendering of complex language structures, far surpassing earlier phrase-based statistical methods. My direct experience with these models confirms their efficiency in maintaining the semantic integrity of the source material, a crucial attribute when dealing with precise content like legal texts.

The BLEU score of 10 from the case study is particularly noteworthy. This score indicates a high level of fluency and accuracy, approaching near-human quality in translation output. Transformer models are defined by their deep learning architecture, which includes numerous parameters fine-tuned during training to capture and reproduce linguistic details accurately. This architecture's ability to continually learn and adapt through training cycles significantly contributes to the improvements in translation quality observed in recent years.